In [9]:
!pip install statsmodels seaborn

In [10]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import os
import sys

current_dir=os.getcwd()
parent_dir=os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from statsmodels.tsa.stattools import adfuller
from models.transformer import EEGDiffusionTransformer
from models.engine import DiffusionEngine
from models.scheduler import DiffusionScheduler
from utils.dataset import EEGDataset

from utils.logger import LoggerHelper

In [14]:
logger=LoggerHelper.get_logger()
base_path = os.getcwd()
adult_path = os.path.join(base_path, "adult", "X_adult.npy")
child_path=os.path.join(current_dir, "pediatric", "X_final.npy")

if os.path.exists(adult_path) and os.path.exists(child_path):
    logger.info("Loading data...")
    logger.info(f"Adult path: {adult_path}")
    logger.info(f"Child path: {child_path}")

    X_adult=np.load(adult_path)
    X_child=np.load(child_path)

    logger.info(f"Adult data loaded: {X_adult.shape}")
    logger.info(f"Child data loaded: {X_child.shape}")
else:
    logger.error(f"Error loading files.")


2025-12-16 19:11:00 - INFO - eeg_pipeline - Loading data...
2025-12-16 19:11:00 - INFO - eeg_pipeline - Adult path: c:\Users\pelin\JupyterNotebookProjects\ADHD\adult\X_adult.npy
2025-12-16 19:11:00 - INFO - eeg_pipeline - Child path: c:\Users\pelin\JupyterNotebookProjects\ADHD\pediatric\X_final.npy
2025-12-16 19:11:01 - INFO - eeg_pipeline - Adult data loaded: (3000, 256, 19)
2025-12-16 19:11:01 - INFO - eeg_pipeline - Child data loaded: (33434, 256, 19)


In [15]:
def check_stationary(signal):
    try:
        result=adfuller(signal, autolag="AIC")
        p_value=result[1]
        return p_value<0.05
    except:
        return False

In [16]:
logger.info("Running stationary test on random 500 samples per group...")
n_samples=500
index_adult=np.random.choice(len(X_adult), n_samples, replace=False)
index_child=np.random.choice(len(X_child), n_samples, replace=False)

adult_stationary_count=0
child_stationary_count=0

test_channels=[0,1]

for channel in test_channels:
    for i in index_adult:
        if check_stationary(X_adult[i, :, channel]):
            adult_stationary_count+=1

    for i in index_child:
        if check_stationary(X_child[i, :, channel]):
            child_stationary_count+=1

total_tests=n_samples*len(test_channels)
adult_ratio=(adult_stationary_count/total_tests)*100
child_ratio=(child_stationary_count/total_tests)*100

stat_gap=adult_ratio-child_ratio

logger.info("Statistical results:")
logger.info(f"Adult stationary rate: %{adult_ratio:.2f} (Source Domain)")
logger.info(f"Child Stationarity Rate:  %{child_ratio:.2f} (Target Domain)")
logger.info(f"Stationary gap: %{stat_gap:.2f}")

if abs(stat_gap)>5:
    logger.info("Conclusion: Significant characteristic diffence detected.")
    logger.info("This scientifically prroves the necessity of transfer learning.")
    


2025-12-16 19:13:03 - INFO - eeg_pipeline - Running stationary test on random 500 samples per group...
2025-12-16 19:13:08 - INFO - eeg_pipeline - Statistical results:
2025-12-16 19:13:08 - INFO - eeg_pipeline - Adult stationary rate: %51.70 (Source Domain)
2025-12-16 19:13:08 - INFO - eeg_pipeline - Child Stationarity Rate:  %62.70 (Target Domain)
2025-12-16 19:13:08 - INFO - eeg_pipeline - Stationary gap: %-11.00
2025-12-16 19:13:08 - INFO - eeg_pipeline - Conclusion: Significant characteristic diffence detected.
2025-12-16 19:13:08 - INFO - eeg_pipeline - This scientifically prroves the necessity of transfer learning.
